In [ ]:
# To obtain the IP address here
!curl ipecho.net/plain

34.90.114.41

In [ ]:
!pip3 install pymongo[srv]

     |████████████████████████████████| 194kB 6.1MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pymongo

# insert_one(), insert_many(), update_one(), update_many(), replace_one(), delete_one(), delete_many()
# The following links are basic 101 to mongoDB x Python, in Chinese.
# https://blog.csdn.net/culiu9261/article/details/107540063
# https://www.runoob.com/python3/python-mongodb.html

my_access_pwd = "FNhkwCvGlAnRcntG"
my_url = "mongodb+srv://saw008:"+ my_access_pwd + "@cluster0.9vbsx.mongodb.net/sampleTable?retryWrites=true&w=majority"
client = pymongo.MongoClient(my_url)

Mounted at /content/drive


下面开始认真了！

In [ ]:
# Get all the databases in my cluster.
print(client.list_database_names())
table_data = client['thesis']['data_table']
table_model = client['thesis']['model_table']
table_perf = client['thesis']['performance_table']

['thesis', 'admin', 'local']


In [ ]:
# To copy files or folders in google drive, you can use the following command.
# !cp -r "/src/folder" "/dest/folder"

#### Functions

In [ ]:
# The following functions is for inserting data_table
import scipy.io as sciio
import pandas as pd
import numpy as np
import csv
import numpy as np
from openpyxl import load_workbook


def load_freq_data(scenario_path):
  folder = scenario_path + '/' if not scenario_path.endswith('/') else scenario_path
  data_file = folder + "PMUData.mat"
  csv_file = folder + "ScenarioDescription.csv"
  mat_file_contents = sciio.loadmat(data_file)
  pmu_locations = mat_file_contents['fmeas_con'][:, 1].astype(int)
  num_time_steps, num_bus = mat_file_contents['PMU']['f'][0][0].shape
  scenario_description = pd.read_csv(csv_file)
  load_changes_start_times = scenario_description['Start time(s) for load changes'][0]
  frequency_data = mat_file_contents['PMU']['f'][0][0][load_changes_start_times * 50 + 25: num_time_steps, pmu_locations - 1]
  return frequency_data

def pattern_obtainer(data):
  precision = 4
  min_val = np.around(np.min(data), decimals=precision)
  max_val = np.around(np.max(data), decimals=precision)
  avg_val = np.around(np.mean(data), decimals=precision)
  std_val = np.around(np.std(data), decimals=precision)
  sample_count = data.shape[0]
  return min_val, max_val, avg_val, std_val, sample_count
  
def entry_dict_data(ifTest, pattern, measurement, dataPath):
  return {"pattern": {
               "min": pattern[0],
               "max": pattern[1],
               "avg": pattern[2],
               "std": pattern[3],
               "no_of_samples": pattern[4]
               }, 
          "measurement": measurement, 
          "data_path": dataPath}

################################################################################

def entry_dict_model(training_data, model_type, model_loc, task_category, training_time, learning_rate, batch_size, epoch_number, applied_libs):
  return {"training_data": training_data,
          "model_type": model_type,
          "model_loc": model_loc,
          "task_category": task_category,
          "training_time": training_time,
          "learning_rate": learning_rate,
          "batch_size": batch_size,
          "epoch_number": epoch_number,
          "applied_libs": applied_libs
          }
def data_distinguisher():
  # read the csv file
  with open("/content/drive/My Drive/myThesis/test001/Testing/FullScenarioGeneration.csv", "r") as f:
    reader = csv.reader(f)
    # extract the skipped scenarios
    temp_list = []
    for item in reader:
        # skip the first line(title)
        if reader.line_num == 1:
            continue
        if item[6] != '--' and item[6] != '':
          tmp = item[6]
          if '\n' in tmp:
            tmp = tmp.replace('\n', ' ')
          if ',' in tmp:
            temp_list = temp_list + [int(x) for x in tmp.split(', ')]
          else:
            temp_list.append(int(item[6]))
  while temp_list[-1] > 450:
    temp_list.pop(-1)
  for i in range(1, 451):
    if (i%15 == 14) or (i%15 == 0):
      temp_list.append(i)
  temp_list=list(set(temp_list))
  # print(temp_list)
  ## 现在不要的scenario是temp_list
  return temp_list
  
def train_data_obtainer(length, inx):
  non_train_data = set(data_distinguisher())
  # 这个samples只是n~n+45
  samples = set(((inx-1)*length+np.array(range(1, length+1))).tolist())
  samples = samples - samples.intersection(non_train_data)
  
  return list(samples)

def read_xlsx(file_now):
    wb = load_workbook(file_now)
    sheets = wb.worksheets   # 获取当前所有的sheet
    sheet1 = sheets[0]
    # val = read_xlsx_cell(sheet1, row, column)
    return sheet1


def read_xlsx_cell(sheet1, row, column):
  val = str(sheet1.cell(row, column).value)
  return val
#   return float((val)[:-1]) if val.endswith('s') else float(val)

def entry_dict_perf(model_id, test_data, task_dict):
  return {
          "model_id": model_id, # TODO
          "test_data": test_data, # str to the path (or ID) of the testing data
          "task": task_dict # Dict
          }

In [ ]:
table_data = client['thesis']['data_table']
table_model = client['thesis']['model_table']
table_perf = client['thesis']['performance_table']

####Data Table

In [ ]:
import time
table_data = client['thesis']['data_table']
temp = []
m = 0

start = time.time()
non_training_data = data_distinguisher()
for i in range(1, 451):
  folder_path = "/content/drive/My Drive/myThesis/test001/Training/scenario" + str(i)
  try:
    this_data = load_freq_data(folder_path)
    # if_test = False if i in non_training_data else True
    temp.append(entry_dict_data(None, pattern_obtainer(this_data), 'frequency', folder_path))
  except:
    # print('sb')
    continue
  
# print(temp)
table_data.insert_many(temp)
end = time.time()
print(end-start)

8.375295400619507


In [ ]:
table_data = client['thesis']['data_table']
table_data.drop()

####Model Table

In [ ]:
# from openpyxl import load_workbook
 
# wb = load_workbook('/content/drive/My Drive/myThesis/test001/Testing/power_10Models.xlsx')
# sheets = wb.worksheets   # 获取当前所有的sheet
# print(sheets)

# # 获取第一张sheet
# sheet1 = sheets[0]
# # sheet1 = wb['Sheet']  # 也可以通过已知表名获取sheet
# print(sheet1)

# # 通过Cell对象读取
# cell_11 = sheet1.cell(2,4).value
# print(cell_11)
# cell_11 = sheet1.cell(1,2).value
# print(cell_11)


# # https://blog.csdn.net/liuyingying0418/article/details/101066630

[<Worksheet "Sheet1">]
<Worksheet "Sheet1">
1174.33s
500-MAE


In [ ]:
# Adding the 10 models
import time
import glob

table_model = client['thesis']['model_table']
temp = []
m = 0

start = time.time()
# model_loc = glob.glob('/content/drive/MyDrive/myThesis/test001/Training/10_trained_models/saved10Model/*.meta')
# for i in range(len(model_loc)):
#   model_loc[i] = model_loc[i].replace('/content/drive/MyDrive/myThesis/test001/Training/10_trained_models/saved10Model', '~')
for i in range(1, 11):
  val = read_xlsx('/content/drive/My Drive/myThesis/test001/Testing/power_10Models.xlsx', 6*(i-1)+2, 4)
  tmp_val = float((val)[:-1]) if val.endswith('s') else float(val)
  tmp_train_data_index_list = train_data_obtainer(45, i)
  tmp_train_data_id_list = []
  model_loc = glob.glob('/content/drive/MyDrive/myThesis/test001/Training/10_trained_models/model'+str(i)+'-*')[0]
  for item in tmp_train_data_index_list:
    tmp_path = '/content/drive/My Drive/myThesis/test001/Training/scenario' + str(item)
    tmp_train_data_id_list.append(table_data.find_one({'data_path': tmp_path})['_id'])
  temp.append(entry_dict_model(tmp_train_data_id_list, "STGCN", model_loc, "prediction", tmp_val, 0.001, 1000, 30, ["TensorFlow"]))
# print(temp)
table_model.insert_many(temp)
end = time.time()
print(end-start)

68.08313608169556


In [ ]:
# Adding the 30 models
import time
import glob

table_model = client['thesis']['model_table']
temp = []
m = 0

start = time.time()
for i in range(1, 31):
  val = read_xlsx('/content/drive/MyDrive/myThesis/test001/Testing/power_30Models.xlsx', 2*i, 9)
  tmp_val = float((val)[:-1]) if val.endswith('s') else float(val)
  tmp_model_id = read_xlsx('/content/drive/MyDrive/myThesis/test001/Testing/power_30Models.xlsx', 2*i, 11)
  tmp_train_data_index_list = train_data_obtainer(15, i)
  tmp_train_data_id_list = []
  model_loc = '/content/drive/MyDrive/myThesis/test001/Training/saved_30models/' + tmp_model_id
  for item in tmp_train_data_index_list:
    tmp_path = '/content/drive/My Drive/myThesis/test001/Training/scenario' + str(item)
    tmp_train_data_id_list.append(table_data.find_one({'data_path': tmp_path})['_id'])
  temp.append(entry_dict_model(tmp_train_data_id_list, "STGCN", model_loc, "prediction", tmp_val, 0.001, 1000, 30, ["TensorFlow"]))
# print(temp)
table_model.insert_many(temp)
end = time.time()
print(end-start)

79.97550439834595


In [ ]:
table_model = client['thesis']['model_table']
table_model.drop()

####Performance Table

In [ ]:
# perf 10 models
import time
import openpyxl

wb = load_workbook('/content/drive/My Drive/myThesis/test001/Testing/power_10Models.xlsx')
sheet1 = wb.worksheets[0]
rows, columns = sheet1.max_row, sheet1.max_column

table_perf = client['thesis']['performance_table']
temp = []

start = time.time()
for i in range(1, rows):
  tmp_model_id = read_xlsx('/content/drive/MyDrive/myThesis/test001/Testing/power_30Models.xlsx', i+1, 11)
  model_id = table_model.find_one({'model_loc': {'$regex': tmp_model_id}})['_id']
  # 

  # obtaining individual ObjectID of test_data
  tmp_test_data_index = int(float(read_xlsx(i+1, 1)))
  tmp_path = '/content/drive/My Drive/myThesis/test001/Training/scenario' + str(tmp_test_data_index)
  test_data = table_data.find_one({'data_path': tmp_path})['_id']

  temp_metric_dict = {'task_category': 'prediction',
                      '500-MAE': float(read_xlsx(i+1, 2)),
                      '500-RMSE': float(read_xlsx(i+1, 3)),
                      'Inference_Time': float((read_xlsx(i+1, 5))[:-1]),
                      'epoch_to_quality': int(float(read_xlsx(i+1, 7)))
                      }
  temp.append(entry_dict_perf(model_id, test_data, temp_metric_dict))

table_perf.insert_many(temp)
end = time.time()
# print(temp)
print(end-start)

17.389697313308716


In [ ]:
# perf 30 models
import time
import openpyxl

wb = load_workbook('/content/drive/My Drive/myThesis/test001/Testing/power_30Models.xlsx')
sheet1 = wb.worksheets[0]
rows, columns = sheet1.max_row, sheet1.max_column
# print(rows)

table_perf = client['thesis']['performance_table']
temp = []
now_sheet = read_xlsx('/content/drive/MyDrive/myThesis/test001/Testing/power_30Models.xlsx')

start = time.time()
for i in range(1, 61):
  tmp_model_id = read_xlsx_cell(now_sheet, i+1, 11)
#   print(i, tmp_model_id)
  model_id = table_model.find_one({'model_loc': {'$regex': tmp_model_id}})['_id']
  # 

  # obtaining individual ObjectID of test_data
  tmp_test_data_index = int(float(read_xlsx_cell(now_sheet, i+1, 2)))
  tmp_path = '/content/drive/My Drive/myThesis/test001/Training/scenario' + str(tmp_test_data_index)
  test_data = table_data.find_one({'data_path': tmp_path})['_id']

  temp_metric_dict = {'task_category': 'prediction',
                      '500-MAE': float(read_xlsx_cell(now_sheet, i+1, 3)),
                      '500-RMSE': float(read_xlsx_cell(now_sheet, i+1, 4)),
                      '300-MAE': float(read_xlsx_cell(now_sheet, i+1, 5)),
                      '300-RMSE': float(read_xlsx_cell(now_sheet, i+1, 6)),
                      '150-MAE': float(read_xlsx_cell(now_sheet, i+1, 7)),
                      '150-RMSE': float(read_xlsx_cell(now_sheet, i+1, 8)),
                      'Inference_Time': float((read_xlsx_cell(now_sheet, i+1, 5))[:-1])
                    #   'epoch_to_quality': int(float(read_xlsx(i+1, 7)))
                      }
  temp.append(entry_dict_perf(model_id, test_data, temp_metric_dict))

table_perf.insert_many(temp)
end = time.time()
# print(temp)
print(end-start)

22.24265146255493


In [ ]:
table_perf = client['thesis']['performance_table']
table_perf.drop()

####尝试查询

In [ ]:
table_model1 = client['thesis']['test_model_table']
table_data1 = client['thesis']['test_data_table']

In [ ]:
myquery = {"pattern.min":  {'$gt': 59.97, '$lt': 59.992}}
haha = table_data1.find(myquery)
print(list(haha))
# def closest(test_query, check_list, num_of_entries):
#     tracer = check_list[0]
#     tmp_lst = list(tracer['pattern'].values())
#     test_q_list = list(test_query['pattern'].values())
#     dist = ((tmp_lst[0]-test_q_list[0])**2 + \
#             (tmp_lst[1]-test_q_list[1])**2 + \
#             (tmp_lst[2]-test_q_list[2])**2 + \
#             (tmp_lst[3]-test_q_list[3])**2) ** 0.5

#     tmp_dict = {tracer['_id']: dist}
    
#     for item in check_list:
#         tmp_lst = list(item['pattern'].values())
#         current_dist = ((tmp_lst[0]-test_q_list[0])**2 + \
#                         (tmp_lst[1]-test_q_list[1])**2 + \
#                         (tmp_lst[2]-test_q_list[2])**2 + \
#                         (tmp_lst[3]-test_q_list[3])**2) ** 0.5
#         if current_dist >= dist:
#             pass
#         else:
#             print(dist)
#             dist = current_dist
#             tracer = item
#     closest_query_id = tracer['_id']
#     return closest_query_id

[{'_id': ObjectId('5fbedc24b6e9aea1ff39d82e'), 'pattern': {'min': 59.9895, 'max': 60.0224, 'avg': 60.0057, 'std': 0.0032, 'no_of_samples': 1375}, 'measurement': 'frequency', 'data_path': '/content/drive/My Drive/myThesis/test001/Training/scenario3'}, {'_id': ObjectId('5fbedc24b6e9aea1ff39d82f'), 'pattern': {'min': 59.9911, 'max': 60.0034, 'avg': 59.9998, 'std': 0.0011, 'no_of_samples': 1375}, 'measurement': 'frequency', 'data_path': '/content/drive/My Drive/myThesis/test001/Training/scenario4'}, {'_id': ObjectId('5fbedc24b6e9aea1ff39d830'), 'pattern': {'min': 59.9845, 'max': 60.0076, 'avg': 59.9968, 'std': 0.0016, 'no_of_samples': 1375}, 'measurement': 'frequency', 'data_path': '/content/drive/My Drive/myThesis/test001/Training/scenario5'}, {'_id': ObjectId('5fbedc24b6e9aea1ff39d832'), 'pattern': {'min': 59.9842, 'max': 60.0169, 'avg': 60.0032, 'std': 0.0025, 'no_of_samples': 1375}, 'measurement': 'frequency', 'data_path': '/content/drive/My Drive/myThesis/test001/Training/scenario7'},

In [ ]:
def closest(test_query, num_of_entries):

    check_list = list(table_data.find({}, {"pattern.min":  1, 
                                           "pattern.max":  1, 
                                           "pattern.avg":  1, 
                                           "pattern.std":  1}))

    test_q_list = list(test_query['pattern'].values())
    tmp_dict = {}
    for item in check_list:
        tmp_lst = list(item['pattern'].values())
        current_dist = ((tmp_lst[0]-test_q_list[0])**2 + \
                        (tmp_lst[1]-test_q_list[1])**2 + \
                        (tmp_lst[2]-test_q_list[2])**2 + \
                        (tmp_lst[3]-test_q_list[3])**2) ** 0.5
        tmp_dict[item['_id']] = current_dist
        
    L = sorted(tmp_dict.items(), key=lambda item:item[1], reverse=False)
    if num_of_entries > 0:
        L = L[:num_of_entries]
    else:
        pass
    dictdata = {}
    for l in L:
        dictdata[l[0]] = l[1]    
    return dictdata


def test_finder(test_query_data, num_of_entries, expected_matches, expected_metric):
    try:
        closest_query_dataID_dist_dict = closest(test_query_data, num_of_entries)
        # Top-N与data_entry相似的data

        query_id_trainData = list(table_model.find({}, {"training_data":  1}))
        modelID_trainData_dict = {}
        for item in query_id_trainData:
            modelID_trainData_dict[list(item.values())[0]] = list(item.values())[1]
        # inter = set(closest_query_dataID_dist_dict.keys()).intersection(set(modelID_trainData_dict.keys()))
        # print(modelID_trainData_dict)
        tmp_matched_models = set()
        for trainDataDict_of_one_model in modelID_trainData_dict:
            tmp_set1 = set(closest_query_dataID_dist_dict.keys())
            tmp_set2 = set(modelID_trainData_dict[trainDataDict_of_one_model])
            if tmp_set1.intersection(tmp_set2):
                tmp_matched_models.add(trainDataDict_of_one_model) # matched modelIDs
                # 所有包含与data_entry相似的data的models
        
        matched_perf = []
        tmp_metric = 'task.' + expected_metric
        perf_entry = table_perf.find({}, {'model_id': 1, tmp_metric: 1})
        for item in tmp_matched_models:
            # tmp_perf_entry = table_perf.find_one({'model_id': item})
            tmp_perf_entry = table_perf.find({"model_id":  item})
            tn = list(tmp_perf_entry)
            # print(tn, len(tn))
            if len(tn) != 0:
                # print(tn)
                matched_perf += tn
        
        new_s = sorted(matched_perf, key = lambda e:e.__getitem__('task').__getitem__(expected_metric))

        return new_s[:expected_matches] # in set/dict format
    except ValueError:
        print('Invalid list, please check again')
        return None

In [ ]:
full_data_table = list(table_data.find({}))
search = full_data_table[6]
# print(search)
print(f"Let's search: {search['pattern']}")
# print(closest(search, -1))
print(test_finder(search, -1, 10, 'Inference_Time'))

Let's search: {'min': 59.9771, 'max': 59.9995, 'avg': 59.9939, 'std': 0.0035, 'no_of_samples': 1375}
[{'_id': ObjectId('5fc52bddf9ad7c82692b9aca'), 'model_id': ObjectId('5fc526cff9ad7c82692b9a92'), 'test_data': ObjectId('5fbeb895e98c620967f2ede7'), 'task': {'task_category': 'prediction', '500-MAE': 0.00020964, '500-RMSE': 0.00032066, '300-MAE': 0.0002755, '300-RMSE': 0.00039364, '150-MAE': 0.00034659, '150-RMSE': 0.00047904, 'Inference_Time': 0.000275}}, {'_id': ObjectId('5fc52bddf9ad7c82692b9acc'), 'model_id': ObjectId('5fc526cff9ad7c82692b9a94'), 'test_data': ObjectId('5fbeb895e98c620967f2ee12'), 'task': {'task_category': 'prediction', '500-MAE': 0.00033437, '500-RMSE': 0.00043366, '300-MAE': 0.00036871, '300-RMSE': 0.0004788, '150-MAE': 0.00036713, '150-RMSE': 0.0005065, 'Inference_Time': 0.0003687}}, {'_id': ObjectId('5fc52bddf9ad7c82692b9acf'), 'model_id': ObjectId('5fc526cff9ad7c82692b9a9e'), 'test_data': ObjectId('5fbeb895e98c620967f2ee1f'), 'task': {'task_category': 'prediction